In [1]:
import cvxpy as cp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import gurobipy

In [2]:
data_dir = '../../output/'

In [3]:
Capacity = 5000
depotname = "MQ"
    
peak_cost = [27.21, 19.50, 5.14]

elec_cost = []

for j in range(48):
    if (j >= 16) & (j < 44):
        elec_cost.append(4.670)
    else:
        elec_cost.append(3.722)

demand = pd.read_csv(data_dir + depotname +'_halfhour_consumption.csv')

demand_day = []
for i in demand.index:
#print(i)
    if demand.Half_Hour[i] < 24:
        demand_day.append(demand.Total[i])
    else:
        demand_day[i-48] += demand.Total[i]
demand_day = np.array(demand_day)  

result = []

Duration = 4 # hr
Power = Capacity / Duration # kW

# In this approach, opmitimizer will try it best to conduct overnight charging.
Depot_Demand2 = cp.Variable(48, nonneg = True) 
Battery_Input2 = cp.Variable(48, nonneg = False) 
SOC2 = cp.Variable(49, nonneg = True) 
Maximum_Demand2 = cp.Variable(3) 

obj2 =  cp.Minimize( peak_cost @ Maximum_Demand2  + 30 * (elec_cost @ Depot_Demand2) / 1000)

con_set2 = []

con_set2.append( Depot_Demand2 == demand_day + Battery_Input2 )
con_set2.append( SOC2 <= i )
con_set2.append( Battery_Input2 <= Power )
con_set2.append( Battery_Input2 >= -Power)

for j in range(48):
    con_set2.append( SOC2[j] + Battery_Input2[j] * 0.5 == SOC2[j+1])
con_set2.append( SOC2[0] == SOC2[48])

for j in range(48):
    if (j >= 16) & (j < 36):
        con_set2.append( Depot_Demand2[j] <= Maximum_Demand2[0] )
    elif (j >= 36) & (j < 44):
        con_set2.append( Depot_Demand2[j] <= Maximum_Demand2[1] )
    else:
        con_set2.append( Depot_Demand2[j] <= Maximum_Demand2[2] )

prob2 = cp.Problem(obj2, con_set2)
prob2.solve(solver = "GUROBI")

result.append(Maximum_Demand2.value[0])
#Result_output(x, demand_day, Depot_Demand2.value, filename = "Cost_" + str(i), depotname = depotname)


Academic license - for non-commercial use only - expires 2022-10-27
Using license file /Users/shentianxiao/gurobi.lic


In [4]:
Capacity = [0, 1000 ,2000 ,5000 ,10000]
x = np.arange(48)/2

result = Depot_opti(Capacity, "MQ")

NameError: name 'Depot_opti' is not defined

In [ ]:
result

In [ ]:
Capacity = [0, 1000 ,2000 ,5000 ,10000]
x = np.arange(48)/2

result = Depot_opti(Capacity, "KB")

In [ ]:
result

In [5]:
Maximum_Demand2.value

array([5406.62456252, 4811.0811202 , 3383.94769114])

In [ ]:
sum( Depot_Demand2 @ elec_cost)

In [6]:
Depot_Demand2.value

array([2892.86561003, 2750.98688232, 2469.71500417, 2246.80333688,
       2039.59116176, 1772.5438897 , 1077.15499102,  914.50342974,
        801.66960043,  774.352521  ,  784.80309654,  678.88692585,
        983.45695965, 1271.92030962, 1907.09340413, 2613.51379408,
       2568.88066926, 4141.14210592, 5187.71207555, 5406.62456252,
       5406.62456252, 4775.03493068, 4812.7433768 , 4341.67287044,
       4115.57779007, 3875.42517928, 4013.74686409, 4009.24811708,
       3903.08375064, 3944.71372862, 3957.45646685, 3988.0616788 ,
       4358.55213864, 4316.30953014, 4474.38060174, 5116.61182172,
       4811.0811202 , 4811.0811202 , 4710.68405845, 4529.14892171,
       4024.43311871, 4106.35023316, 3795.01726828, 3888.42492931,
       3383.94769114, 3383.94769114, 3086.19091391, 2976.2270865 ])

In [7]:
elec_cost @ Depot_Demand2.value

711341.9449147738

In [8]:
elec_cost

[3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 3.722,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 4.67,
 3.722,
 3.722,
 3.722,
 3.722]

In [9]:
peak_cost @ Maximum_Demand2.value

258323.8273225867

In [12]:
Battery_Input2.value@Battery_Input2.value

124397.03759935795